In [8]:
import requests
from bs4 import BeautifulSoup

# Function to scrape text from a website
def scrape_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Modify this based on the structure of the webpage to extract relevant text
    text = ' '.join([p.text for p in soup.find_all('p')])
    return text

# Define URLs of Arabic websites related to your topic
urls = [
    'https://alkhalilarabic.com/',
    'https://guidetoarabic.net/ar'
]

# Define a function to calculate text score based on some criteria
def calculate_text_score(text):
    return len(text) / 1000  # Just a simple scaling factor for demonstration

# Create a dictionary to store text and corresponding scores
data = {}
texts =[]
scores =[]

# Scrape text from each URL and calculate the score
for url in urls:
    text = scrape_text(url)
    texts.append(text)
    score = calculate_text_score(text)
    scores.append(score)
    data[url] = {'text': text, 'score': score}

# Print the data
for url, info in data.items():
    print(f"URL: {url}")
    print(f"Text:\n{info['text']}")
    print(f"Score: {info['score']}\n")
    
print(texts)
print(scores)

URL: https://alkhalilarabic.com/
Text:
الخليل لتعليم اللغة العربية اكتسب المهارات اللغوية وتعرف على الثقافة العربية لتفتح عالمك مع منصة الخليل. تعلم العربية في دورات عن بعد على أيدي خبراء انطلق في تعلم العربية وصقل مهاراتك اللغوية في القراءة والكتابة والمحادثة والاستماع من خلال الدورات الأساسية على منصة الخليل تعلم اللغة العربية من الخبراء في اللغة الناطقين بالعربية، وانغمس في ثقافتها، وتعرف على قيمها وحضارتها. تعلم اللغة العربية باحترافية وطوّر مهاراتك اللغوية عبر منصتنا التعليمية المتطورة وأنت في أي موقع من العالم. بإمكانك ممارسة وتطوير مهارة المحادثة لساعات إضافية مع أفضل المدرسين العرب، وفي الأوقات التي تناسبك. اكتسب مهارات لغوية جديدة ترتبط بمجال عملك أو دراستك أو اهتمامك اللغوي في المجالات الدبلوماسية أو الإعلامية أو الأكاديمية أو غيرها أحد مشاريع شركة مؤتلف (إبانة)
Score: 0.743

URL: https://guidetoarabic.net/ar
Text:
اللغة العربية هي لغة القرآن الكريم، وهي وسيلة لفهم النصوص الشرعية، والاستنباط الصحيح من النصوص، وتعلم اللغة العربية واجب على المسلمين لفهم القرآن الكريم والسنة الن

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Assuming 'text_data' contains our preprocessed text data
texts = [info['text'] for info in data.values()]
scores = [info['score'] for info in data.values()]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding sequences to ensure uniform length
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Converting scores to numpy array
labels = np.array(scores)


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, GRU
from sklearn.model_selection import train_test_split


# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# RNN Model
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len))
rnn_model.add(LSTM(64))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Bidirectional RNN Model
bidirectional_rnn_model = Sequential()
bidirectional_rnn_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len))
bidirectional_rnn_model.add(Bidirectional(LSTM(64)))
bidirectional_rnn_model.add(Dense(1, activation='sigmoid'))
bidirectional_rnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
bidirectional_rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# GRU Model
gru_model = Sequential()
gru_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len))
gru_model.add(GRU(64))
gru_model.add(Dense(1, activation='sigmoid'))
gru_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.0610 - mae: 0.2471 - val_loss: 1.1195 - val_mae: 1.0581
Epoch 2/10
1/1 [==============================] - 0s 137ms/step - loss: 0.0569 - mae: 0.2384 - val_loss: 1.1138 - val_mae: 1.0554
Epoch 3/10
1/1 [==============================] - 0s 151ms/step - loss: 0.0528 - mae: 0.2298 - val_loss: 1.1077 - val_mae: 1.0525
Epoch 4/10
1/1 [==============================] - 0s 137ms/step - loss: 0.0488 - mae: 0.2208 - val_loss: 1.1010 - val_mae: 1.0493
Epoch 5/10
1/1 [==============================] - 0s 128ms/step - loss: 0.0445 - mae: 0.2110 - val_loss: 1.0933 - val_mae: 1.0456
Epoch 6/10
1/1 [==============================] - 0s 104ms/step - loss: 0.0400 - mae: 0.2000 - val_loss: 1.0845 - val_mae: 1.0414
Epoch 7/10
1/1 [==============================] - 0s 125ms/step - loss: 0.0351 - mae: 0.1874 - val_loss: 1.0740 - val_mae: 1.0364
Epoch 8/10
1/1 [==============================] - 0s 120ms/step - loss: 0.0298 - mae: 0.1726 

In [16]:
# Evaluate the models
rnn_scores = rnn_model.evaluate(X_test, y_test)
bidirectional_rnn_scores = bidirectional_rnn_model.evaluate(X_test, y_test)
gru_scores = gru_model.evaluate(X_test, y_test)

print("RNN Model Evaluation:")
print("Mean Squared Error:", rnn_scores[0])
print("Mean Absolute Error:", rnn_scores[1])

print("\nBidirectional RNN Model Evaluation:")
print("Mean Squared Error:", bidirectional_rnn_scores[0])
print("Mean Absolute Error:", bidirectional_rnn_scores[1])

print("\nGRU Model Evaluation:")
print("Mean Squared Error:", gru_scores[0])
print("Mean Absolute Error:", gru_scores[1])


1/1 [==============================] - 0s 31ms/step - loss: 1.0521 - mae: 1.0257
RNN Model Evaluation:
Mean Squared Error: 1.0269255638122559
Mean Absolute Error: 1.0133733749389648

Bidirectional RNN Model Evaluation:
Mean Squared Error: 1.0106168985366821
Mean Absolute Error: 1.0052944421768188

GRU Model Evaluation:
Mean Squared Error: 1.052139163017273
Mean Absolute Error: 1.0257383584976196
